In [1]:
# Call your prediction function
from typing import List
import numpy as np
import cv2
import uvicorn
from PIL import Image, ImageDraw
# import boto3
from datetime import datetime
import random
import string
import os

from ultralytics import YOLO
glass_model = YOLO('models/glass_nugget.pt')
# drink_model = YOLO('/home/user1/Yolov8/Modified Data/runs/segment/pc_trained_emptyglass/weights/last.pt')
# drink_model = YOLO('/home/user1/Downloads/overfit_ilast.pt')
drink_model = YOLO('models/augment_best_drink.pt')
def predict_image(original_image) -> str:
    # Run glass detection on the image
    glass_results = glass_model.predict(original_image, save=False, conf=0.55, device='cpu', save_txt=False)
    # Process each detected glass in the image
    boxes = glass_results[0].boxes.xyxy.tolist()
    classes = glass_results[0].boxes.cls.tolist()
    names = glass_results[0].names
    confidences = glass_results[0].boxes.conf.tolist()
    # count_glass=len(confidences)
    if len(confidences) == len(boxes):
        count_glass = len(confidences)
    refill_list = []
    cropped_glass_pil=None
    # Iterate through detected glasses based on classes and bounding boxes
    for box, cls_id, conf in zip(boxes, classes, confidences):
        if names[cls_id] == 'Glass':  # Adjust this if your class name for glasses is different
            # Extract bounding box coordinates for the glass
            gx1, gy1, gx2, gy2 = map(int, box)

            # Calculate the height of the glass
            glass_height = gy2 - gy1

            # Crop the detected glass area from the original image
            cropped_glass = original_image[gy1:gy2, gx1:gx2]

            # Convert cropped area to the format expected by YOLO (PIL Image)
            cropped_glass_pil = Image.fromarray(cv2.cvtColor(cropped_glass, cv2.COLOR_BGR2RGB))
            # cropped_glass_pil.show()
            # Detect drink in the cropped glass area
            drink_results = drink_model.predict(cropped_glass_pil, conf=0.5, device='cpu', save=False)
            # print(drink_results[0].boxes.xyxy)
            # Check if there is any drink detected within the glass
            if len(drink_results[0].boxes.xyxy) > 0:  # Check if any detections
                # Extract bounding box coordinates for the drink
                dx1, dy1, dx2, dy2 = map(int, drink_results[0].boxes.xyxy[0])
                draw = ImageDraw.Draw(cropped_glass_pil)

                # Define bounding box coordinates
                bbox = [(dx1, dy1), (dx2, dy2)]

                # Draw bounding box rectangle
                draw.rectangle(bbox, outline="red", width=2)

                # Display the image with bounding box
                # cropped_glass_pil.show()

                #half_glass = dy2 // 2.5
                half_glass = dy2 * (1 - 0.50)
       
                if half_glass < dy1:
                    refill_list.append("Refill")
                else:
                    refill_list.append("No")
            else:
                refill_list.append("Refill")
    if "Refill" in refill_list:
        return "Refill",cropped_glass_pil
    elif "No" in refill_list:
        return "No Refill",cropped_glass_pil
    elif not refill_list:
        return "No Glass",cropped_glass_pil


In [4]:
import pandas as pd
data=pd.read_csv('data.csv')
data.head()
# images=data['image_path']
images=os.listdir('IMAGES_test')
labels=data['label']
for i,img in enumerate(images):
    image=cv2.imread('IMAGES_test/'+img)
    print(image)
    prediction, returned_image = predict_image(image)
    print(img, prediction, labels[i])
    # if prediction!=labels[i]:
    #     if returned_image is not None:
    #         print("asfafasfaf")
    #         img2 = cv2.cvtColor(np.array(returned_image), cv2.COLOR_RGB2BGR)

    #         # Resize images to the same height if they differ (optional)
    #         height1, width1, channels1 = image.shape
    #         height2, width2, channels2 = img2.shape

    #         if height1 != height2:
    #             # Resize both images to the height of the first image
    #             image = cv2.resize(image, (int(width1 * height1 / height1), height1))
    #             img2 = cv2.resize(img2, (int(width2 * height1 / height2), height1))

    #         # Concatenate the images side by side
    #         concatenated_image = np.hstack((image, img2))

    #         # Add titles
    #         font = cv2.FONT_HERSHEY_SIMPLEX
    #         cv2.putText(concatenated_image, 'Original', (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    #         cv2.putText(concatenated_image, 'Prediction with Bounding Box', (width1 + 10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

    #         # Display the image (for debugging purposes)
    #         cv2.imshow('Concatenated Image', concatenated_image)
    #         cv2.waitKey(0)
    #         cv2.destroyAllWindows()
    # if prediction != "No Glass":
    #     #Upload the image to s3.
    #     dt_string = datetime.now().strftime("%Y-%m-%d_%H %M %S")
    #     rand_string = ''.join(random.choice(string.digits) for i in range(5))

    #     output_filename = f'./{prediction}_{dt_string}_{rand_string}.jpg'

    #     if cv2.imwrite(output_filename, img):
    #         print("Image saved.")
    #     else:
    #         print("Image could not be saved.")

    #     # bucket.upload_file(output_filename, output_filename[2:])
    #     os.remove(output_filename)

    # return {"prediction": prediction}

[[[200 199 161]
  [200 199 161]
  [199 199 163]
  ...
  [170 168 168]
  [166 164 163]
  [166 164 163]]

 [[200 199 161]
  [199 200 161]
  [199 199 163]
  ...
  [171 169 169]
  [171 169 169]
  [170 171 169]]

 [[199 200 161]
  [199 200 161]
  [199 199 163]
  ...
  [167 166 168]
  [165 164 166]
  [162 164 164]]

 ...

 [[  9  24  16]
  [ 11  27  16]
  [  8  25  12]
  ...
  [ 11  24  10]
  [  9  24  10]
  [  9  24  10]]

 [[  9  23  17]
  [ 10  25  17]
  [  7  24  11]
  ...
  [  9  25   8]
  [ 10  25  11]
  [ 10  25  11]]

 [[ 11  25  19]
  [ 12  27  19]
  [  9  25  14]
  ...
  [ 10  26   9]
  [ 10  25  11]
  [ 10  25  11]]]
